In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/train.csv").drop("~", axis = 1) #경로지정
test = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/test.csv").drop("No", axis = 1)
sample_sub = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/submission.csv")

In [3]:
def preprocessing(data):
    # Lung cancer TNM stage
    #T
    data.loc[(data['M1a'] == 1 ) | (data['M1b'] == 1) | (data['M1c'] == 1), 'M0' ] = 0 
    data.loc[(data['M1a'] == 0) & (data['M1b'] == 0) & (data['M1c'] == 0),'M0'] = 1

    #N
    data.loc[(data['N1'] == 1 ) | (data['N2'] == 1) | (data['N3'] == 1), 'N0'] = 0
    data.loc[(data['N1'] == 0) & (data['N2'] == 0) & (data['N3'] == 0), 'N0'] = 1
    #M
    
    #stage
    data['TNM_stage'] = ''

    stage_map = {

        ('T1', 'N0'): 'IA', ('T1', 'N1'): 'IIA', ('T1', 'N2'): 'IIB', ('T1', 'N3'): 'IIIA', 
        ('T1a', 'N0'): 'IA', ('T1a', 'N1'): 'IIA', ('T1a', 'N2'): 'IIB', ('T1a', 'N3'): 'IIIA',
        ('T1b', 'N0'): 'IA', ('T1b', 'N1'): 'IIA', ('T1b', 'N2'): 'IIB', ('T1b', 'N3'): 'IIIA', 
        ('T1c', 'N0'): 'IA', ('T1c', 'N1'): 'IIA', ('T1c', 'N2'): 'IIB', ('T1c', 'N3'): 'IIIA', 
        ('T2', 'N0'): 'IB', ('T2', 'N1'): 'IIB', ('T2', 'N2'): 'IIIA', ('T2', 'N3'): 'IIIB',
        ('T2a', 'N0'): 'IB', ('T2a', 'N1'): 'IIB', ('T2a', 'N2'): 'IIIA', ('T2a', 'N3'): 'IIIB', 
        ('T2b', 'N0'): 'IIA', ('T2b', 'N1'): 'IIB', ('T2b', 'N2'): 'IIIA', ('T2b', 'N3'): 'IIIB', 
        ('T3', 'N0'): 'IIB', ('T3', 'N1'): 'IIIA', ('T3', 'N2'): 'IIIA', ('T3', 'N3'): 'IIIB',
        ('T4', 'N0'): 'IIIA', ('T4', 'N1'): 'IIIA', ('T4', 'N2'): 'IIIB', ('T4', 'N3'): 'IIIC'
    }

    for t in ['T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4']:
        for n in ['N0', 'N1', 'N2', 'N3']:
            mask = (data[t] == 1) & (data[n] == 1)
            target = stage_map.get((t, n), 'Invalid TNM')
            data.loc[mask, 'TNM_stage'] = target
            if target == 'Invalid TNM':
                print((t, n))

    data.loc[data['M1a'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1b'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1c'] == 1, 'TNM_stage'] = 'IVB'

    stage_encode = {'' : 0, 'IA' : 1, "IB" : 2, "IIA" : 3, "IIB" : 4 , "IIIA" : 5, "IIIB" : 6, "IIIC": 7, "IVA": 8, "IVB":9}
    stage_encode2 = {0 : 0, 1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4}
    data['TNM_stage'] = data['TNM_stage'].map(stage_encode)
    data['TNM_bigstage'] = data['TNM_stage'].map(stage_encode2)
    
    # Surgery
    data['Therapy_count'] = data['Operation'] + data['Chemotherapy'] + data['Radiation Therapy']

    # Other
    data['BMI'] = data['Weight'] / (data['Height'] / 100) ** 2

    # EGFR mutation Detection encoding 99 -> 0
    data['EGFR mutation Detection'] = data['EGFR mutation Detection'].replace(99, 2)

    # Type of Drink encoding 99 -> 0
    data['Type of Drink'] = data['Type of Drink'].replace(99, 0)

    # Mu
    return data

In [4]:
train = preprocessing(train)
test = preprocessing(test)

# XGBOOST

In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X = train.drop("Death", axis = 1)
y = train["Death"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

xgb = XGBClassifier(n_estimators = 900, learning_rate = 0.02, max_depth = 5, random_state = 2)
xgb.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = xgb.predict(X_val)
print(accuracy_score(y_val, y_pred))
confusion_matrix(y_val, y_pred)

0.782


array([[1562,   11],
       [ 425,    2]])

In [5]:
from xgboost import XGBClassifier

X = train.drop("Death", axis = 1)
y = train["Death"]

xgb = XGBClassifier(n_estimators = 900, learning_rate = 0.02, max_depth = 5, random_state = 2)
xgb.fit(X, y)
print(sum(xgb.predict(test)))

34


In [6]:
pred = xgb.predict(test)
sample_sub["Death"] = pred
sample_sub["Death"].value_counts()

Death
0    4966
1      34
Name: count, dtype: int64

In [44]:
sample_sub.to_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/SUBMISSION/5_15_2.csv", index = False)